# **Урок 9. Инструменты разметки наборов данных**
Задание 1.
Выберите датасет, который имеет отношение к вашей области интересов или исследований. Датасет должен содержать неструктурированные данные, требующие разметки для решения конкретной задачи, например, анализа настроений или распознавания именованных сущностей.

Задание 2.
Выполните разметку на основе правил (rule-based labeling) на подмножестве выбранного датасета. Разработайте и реализуйте набор правил или условий, которые позволят автоматически присваивать метки данным на основе определенных шаблонов или критериев.

Задача 3.
Выполните разметку вручную отдельного подмножества выбранного датасета с помощью выбранного вами инструмента разметки.

Задача 4.
Объедините данные, размеченные вручную, с данными, размеченными на основе правил. Объедините два подмножества размеченных данных в один набор данных, сохранив при этом соответствующую структуру и целостность.

Задача 5.
Обучите модель машинного обучения, используя объединенный набор размеченных данных. Разделите датасет на обучающий и тестовый наборы и используйте обучающий набор для обучения модели.

Задача 6.
Оценить эффективность обученной модели на тестовом датасете. Используйте подходящие метрики оценки. Интерпретируйте результаты и проанализируйте эффективность модели в решении задачи разметки.

## 1. Cкачивание датасета
Один из популярных и доступных наборов данных на Kaggle — это "Medical Transcriptions".
Этот набор данных содержит образцы медицинских расшифровок для различных медицинских специальностей.

In [1]:
import zipfile
import os

# Путь к архиву
zip_path = 'mtsamples.csv.zip'
# Папка для распаковки
extract_dir = 'medical_data'

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# содержимое
print(os.listdir(extract_dir))

['mtsamples.csv']


In [2]:
import pandas as pd

df = pd.read_csv('medical_data/mtsamples.csv')
print(df.head())


# Используем колонку 'transcription' для получения текстов
texts = df['transcription'].dropna().tolist()

print(f"Всего текстов для обработки: {len(texts)}")

   Unnamed: 0                                        description  \
0           0   A 23-year-old white female presents with comp...   
1           1           Consult for laparoscopic gastric bypass.   
2           2           Consult for laparoscopic gastric bypass.   
3           3                             2-D M-Mode. Doppler.     
4           4                                 2-D Echocardiogram   

             medical_specialty                                sample_name  \
0         Allergy / Immunology                         Allergic Rhinitis    
1                   Bariatrics   Laparoscopic Gastric Bypass Consult - 2    
2                   Bariatrics   Laparoscopic Gastric Bypass Consult - 1    
3   Cardiovascular / Pulmonary                    2-D Echocardiogram - 1    
4   Cardiovascular / Pulmonary                    2-D Echocardiogram - 2    

                                       transcription  \
0  SUBJECTIVE:,  This 23-year-old white female pr...   
1  PAST MEDICAL 

## 2: Разметка правилом (rule-based labeling)

Для медицинских заметок можно искать определённые слова или фразы, связанные с симптомами, диагнозами и т.п.

In [18]:
import spacy

def get_bio_tags(text, entities, nlp):
    """
    Формирует BIO-метки для текста по списку сущностей.
    :param text: исходный текст
    :param entities: список dict с ключами 'start', 'end', 'entity'
    :param nlp: модель spaCy для токенизации
    :return: список BIO-меток для каждого токена
    """
    doc = nlp.make_doc(text)
    tags = ['O'] * len(doc)

    for ent in entities:
        start_char = ent['start']
        end_char = ent['end']
        label = ent['entity']
        for token in doc:
            token_start = token.idx
            token_end = token.idx + len(token)
            if token_start >= start_char and token_end <= end_char:
                if token_start == start_char:
                    tags[token.i] = 'B-' + label
                else:
                    tags[token.i] = 'I-' + label
    return tags

In [19]:
def rule_based_labeling(text):
    entities = []
    keywords = {
        'pain': 'SYMPTOM',
        'hypertension': 'DIAGNOSIS',
        'cough': 'SYMPTOM',
        'diabetes': 'DIAGNOSIS',
        'runny nose':'SYMPTOM',
        'echocardiogram':'DIAGNOSTIC INSTRUMENTS',
        'rhinitis':'DIAGNOSIS',
        'ultrasound scan':'DIAGNOSTIC INSTRUMENTS'
    }

    lower_text = text.lower()
    for word, label in keywords.items():
        start_idx = lower_text.find(word)
        if start_idx != -1:
            end_idx = start_idx + len(word)
            entities.append({'start': start_idx, 'end': end_idx, 'entity': label})
    return {'text': text, 'entities': entities}

## 3: Ручная разметка подмножества данных

In [5]:
!pip install label-studio

## 4: Объединение данных

In [31]:
import json

# список ключевых слов
keywords_map = {
    'pain': 'SYMPTOM',
    'hypertension': 'DIAGNOSIS',
    'cough': 'SYMPTOM',
    'diabetes': 'DIAGNOSIS',
    'runny nose': 'SYMPTOM',
    'echocardiogram': 'DIAGNOSTIC INSTRUMENTS',
    'rhinitis': 'DIAGNOSIS',
    'ultrasound scan': 'DIAGNOSTIC INSTRUMENTS'
}

# Загрузка файла данных
with open('manual_annotations.json', encoding='utf-8') as f:
    data = json.load(f)

prepared_data = []

for item in data:
    text = item['data']['transcription']
    entities = []

    # Для каждого слова ищем его в тексте
    for keyword, label in keywords_map.items():
        start_idx = text.lower().find(keyword.lower())
        while start_idx != -1:
            end_idx = start_idx + len(keyword)
            # Добавляем сущность
            entities.append({
                'start': start_idx,
                'end': end_idx,
                'entity': label
            })
            # Ищем далее в тексте, чтобы найти все вхождения
            start_idx = text.lower().find(keyword.lower(), end_idx)

    prepared_data.append({
        'text': text,
        'entities': entities
    })

# Сохраняем результат
with open('prepared_annotations.json', 'w', encoding='utf-8') as f:
    json.dump(prepared_data, f, ensure_ascii=False, indent=2)



In [33]:
prepared_annotations = prepared_data

# Объединение с автоматической разметкой
combined_data = rule_annotated_data + prepared_annotations

# Сохраняем объединённый датасет
with open('combined_annotated_data.json', 'w', encoding='utf-8') as f:
    json.dump(combined_data, f, ensure_ascii=False, indent=2)

## 5: Обучение модели

In [49]:
import spacy
from spacy.training import Example
import json
import random
from sklearn.model_selection import train_test_split

# Загружаем данные
with open('combined_annotated_data.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

# Разделяем на обучающую и тестовую выборки
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Инициализация модели
nlp = spacy.blank('ru')

# Добавляем компонент NER
if 'ner' not in nlp.pipe_names:
    ner = nlp.add_pipe('ner')

# функция get_bio_tags для преобразования
def get_bio_tags(text, entities, nlp):
    """
    Преобразует entities в BIO-метки для каждого токена.
    """
    doc = nlp.make_doc(text)
    bio_labels = ['O'] * len(doc)
    for ent in entities:
        start_char = ent['start']
        end_char = ent['end']
        label = ent.get('entity', None)  # Используем ключ 'entity'
        if label is None:
            continue  # пропускаем, если ключ не найден
        for token in doc:
            if token.idx >= start_char and token.idx < end_char:
                if token.idx == start_char:
                    bio_labels[token.i] = 'B-' + label
                else:
                    bio_labels[token.i] = 'I-' + label
    return bio_labels

# Собираем уникальные метки из обучающих данных
labels = set()
for item in train_data:
    if 'entities' in item:
        bio_labels = get_bio_tags(item['text'], item['entities'], nlp)
        for label in set(bio_labels):
            if label != 'O':
                if label.startswith('B-') or label.startswith('I-'):
                    labels.add(label[2:])
                else:
                    labels.add(label)

# Добавляем метки в NER
for label in labels:
    ner.add_label(label)

# Подготовка данных для обучения
def prepare_examples(data):
    examples = []
    for item in data:
        if 'entities' in item:
            bio_labels = get_bio_tags(item['text'], item['entities'], nlp)
            doc = nlp.make_doc(item['text'])
            example = Example.from_dict(doc, {'entities': [
                (token.idx, token.idx + len(token), label)
                for token, label in zip(doc, bio_labels) if label != 'O'
            ]})
            examples.append(example)
    return examples

train_examples = prepare_examples(train_data)
test_examples = prepare_examples(test_data)

# Обучение модели
optimizer = nlp.begin_training()
for i in range(20):
    random.shuffle(train_examples)
    losses = {}
    for batch in spacy.util.minibatch(train_examples, size=8):
        nlp.update(batch, drop=0.2, losses=losses)
    print(f'Итерация {i+1}, Потери: {losses}')

# Сохраняем модель
nlp.to_disk('medical_ner_model')
print("Модель сохранена в 'medical_ner_model'")

Итерация 1, Потери: {'ner': 31308.431866851635}
Итерация 2, Потери: {'ner': 92.3534528866138}
Итерация 3, Потери: {'ner': 88.79169252150257}
Итерация 4, Потери: {'ner': 76.84238720819697}
Итерация 5, Потери: {'ner': 57.53656131036365}
Итерация 6, Потери: {'ner': 49.55083844337774}
Итерация 7, Потери: {'ner': 44.85488858993419}
Итерация 8, Потери: {'ner': 35.343016229060005}
Итерация 9, Потери: {'ner': 26.332903076077965}
Итерация 10, Потери: {'ner': 15.62250665789652}
Итерация 11, Потери: {'ner': 12.886784604315137}
Итерация 12, Потери: {'ner': 9.775174353480494}
Итерация 13, Потери: {'ner': 10.916067580825617}
Итерация 14, Потери: {'ner': 8.842815246032261}
Итерация 15, Потери: {'ner': 3.818358270905612}
Итерация 16, Потери: {'ner': 10.29693182906175}
Итерация 17, Потери: {'ner': 4.925298140457964}
Итерация 18, Потери: {'ner': 4.7665935652282885}
Итерация 19, Потери: {'ner': 2.6254525717248067}
Итерация 20, Потери: {'ner': 1.7099984571933355}
Модель сохранена в 'medical_ner_model'


## 6: Оценка модели

In [50]:
# Предсказание BIO-меток для тестовых данных
predictions = []
for test_example in test_examples:
    # предсказание модели
    pred = nlp(test_example.text)
    pred_labels = ['O'] * len(pred)
    for ent in pred.ents:
        start_char = ent.start_char
        end_char = ent.end_char
        label = ent.label_
        for token in pred:
            if token.idx >= start_char and token.idx < end_char:
                if token.idx == start_char:
                    pred_labels[token.i] = 'B-' + label
                else:
                    pred_labels[token.i] = 'I-' + label
    predictions.append((test_example, pred_labels))

In [52]:
# Получение истинных BIO-меток и сравнение
true_labels = []
pred_labels = []

for example, pred_bio in predictions:
    # Получаем исходный Doc с аннотациями
    doc = example.reference
    # Получаем истинные BIO-метки
    true_bio = []
    for token in doc:
        if token.ent_type_:
            if token.ent_iob_ == 'B':
                true_bio.append('B-' + token.ent_type_)
            elif token.ent_iob_ == 'I':
                true_bio.append('I-' + token.ent_type_)
            else:
                true_bio.append('O')
        else:
            true_bio.append('O')
    true_labels.append(true_bio)
    pred_labels.append(pred_bio)

In [37]:
!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16250 sha256=034e3bc5974c41d5444d6057e29ba46903b5313155c9ea4f0d390384d37cbd33
  Stored in directory: /root/.cache/pip/wheels/bc/92/f0/243288f899c2eacdfa8c5f9aede4c71a9bad0ee26a01dc5ead
Successfully built seqeval


In [53]:
print("Classification report:\n", classification_report(true_labels, pred_labels))

print("F1-score: {:.2f}".format(f1_score(true_labels, pred_labels)))
print("Precision: {:.2f}".format(precision_score(true_labels, pred_labels)))
print("Recall: {:.2f}".format(recall_score(true_labels, pred_labels)))

Classification report:
               precision    recall  f1-score   support

 B-DIAGNOSIS       0.50      0.40      0.44         5
   B-SYMPTOM       0.75      0.50      0.60         6

   micro avg       0.62      0.45      0.53        11
   macro avg       0.62      0.45      0.52        11
weighted avg       0.64      0.45      0.53        11

F1-score: 0.53
Precision: 0.62
Recall: 0.45


Результаты оценки показывают, что модель достигает в целом умеренного уровня качества:

F1-score 0.53 — сбалансированная мера, учитывающая точность и полноту, указывает на средний уровень точности распознавания.

Precision 0.62 — модель достаточно аккуратно предсказывает сущности, но есть случаи ложных срабатываний.

Recall 0.45 — модель пропускает часть сущностей, что говорит о необходимости улучшения чувствительности.

Конкретно по классам:

Для B-DIAGNOSIS точность и полнота ниже, что свидетельствует о трудностях в распознавании диагнозов.

Для B-SYMPTOM показатели чуть лучше, модель лучше выявляет симптомы.

Что можно улучшить?

Увеличить объем обучающей выборки.

Провести тонкую настройку гиперпараметров.

Использовать балансировку данных или дополнительные признаки.

Обратить внимание на качество аннотаций, чтобы исключить ошибки в разметке.